In [10]:
import numpy as np
import pandas as pd
import bloscpack as bp

from sklearn.model_selection import StratifiedKFold

import imgaug as ia
import imgaug.augmenters as iaa

from torch.utils.data.dataloader import DataLoader

import fastai
from fastai.vision import *
from fastai.callbacks import *

from optim import Over9000
from data import Bengaliai_DS
from models_mg import Seresnext50MishSumSpatial
from callback_utils import SaveModelCallback
from mixup_fastai_utils import CmCallback, MuCmCallback, MixUpCallback
from loss import Loss_combine_weighted_v2
from metric import Metric_grapheme, Metric_vowel, Metric_consonant, Metric_tot

In [1]:
from torchvision.models import resnet34

In [2]:
a = resnet34()

In [16]:
a = torch.nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

In [19]:
a(arr).shape

torch.Size([10, 64, 64, 64])

In [6]:
from models_mg import Res34Complex

In [7]:
b = Res34Complex()

In [11]:
arr = torch.from_numpy(np.random.rand(10, 3, 128, 128)).float()

In [13]:
c = b.feature_extractor(arr)

In [14]:
c.shape

torch.Size([10, 512, 16, 16])

---

In [2]:
SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

In [1]:
from senet_mod import se_resnext50_32x4d

In [2]:
a = se_resnext50_32x4d()

In [4]:
import numpy as np
import torch

In [8]:
b = a.features(torch.from_numpy(np.random.rand(10, 3, 224, 224)).float())

In [9]:
b.shape

torch.Size([10, 2048, 7, 7])

---
### data

In [3]:
pdf = pd.read_csv('../input/train.csv')
unique_grapheme = pdf['grapheme'].unique()
grapheme_code = dict([(g, c) for g, c in zip(unique_grapheme, np.arange(unique_grapheme.shape[0]))])
pdf['grapheme_code'] = [grapheme_code[g] for g in pdf['grapheme']]

skf = StratifiedKFold(n_splits=7, shuffle=True, random_state=42)
for trn_ndx, vld_ndx in skf.split(pdf['grapheme_code'], pdf['grapheme_code']):
    break
    
imgs = bp.unpack_ndarray_from_file('../features/train_images_size128_pad0_max.bloscpack')
lbls = pd.read_csv('../input/train.csv').iloc[:, 1:4].values

trn_imgs = imgs[trn_ndx]
trn_lbls = lbls[trn_ndx]
vld_imgs = imgs[vld_ndx]
vld_lbls = lbls[vld_ndx]

In [4]:
# augs = iaa.SomeOf(
#     (0, 2),
#     [
#         iaa.OneOf(
#             [
#                 iaa.Affine(scale={"x": (0.8, 1.), "y": (0.8, 1.)}, rotate=(-15, 15), shear=(-15, 15)),
#                 iaa.PerspectiveTransform(scale=.08, keep_size=True),
#             ]
#         ),
#         iaa.PiecewiseAffine(scale=(.02, .03)),
#     ],
#     random_order=True
# )

augs = iaa.SomeOf(
    (1, 3),
    [
        iaa.Affine(scale={"x": (0.8, 1.), "y": (0.8, 1.)}, rotate=(-10, 10), shear=(-10, 10)),
        iaa.PiecewiseAffine(scale=(0.02, 0.03)),
        iaa.DirectedEdgeDetect(alpha=(.01, .99), direction=(0.0, 1.0)),
    ],
    random_order=True
)

In [5]:
batch_size = 64 # 64 is important as the fit_one_cycle arguments are probably tuned for this batch size

training_set = Bengaliai_DS(trn_imgs, trn_lbls, transform=augs)
validation_set = Bengaliai_DS(vld_imgs, vld_lbls)

training_loader = DataLoader(training_set, batch_size=batch_size, num_workers=6, shuffle=True) # , sampler=sampler , shuffle=True
validation_loader = DataLoader(validation_set, batch_size=batch_size, num_workers=6, shuffle=False)

data_bunch = DataBunch(train_dl=training_loader, valid_dl=validation_loader)

---
### model

In [6]:
device = 'cuda:0'
n_grapheme = 168
n_vowel = 11
n_consonant = 7
n_total = n_grapheme + n_vowel + n_consonant

In [7]:
classifier = Seresnext50MishSumSpatial()

In [8]:
learn = Learner(
    data_bunch,
    classifier,
    loss_func=Loss_combine_weighted_v2(),
    opt_func=Over9000,
    metrics=[Metric_grapheme(), Metric_vowel(), Metric_consonant(), Metric_tot()]
)

logger = CSVLogger(learn, 'new_baseline_Seresnext50MishSumSpatial_lastBN_LessAugs_211_Mu10_Wd0_64epochs')

learn.unfreeze()

In [ ]:
learn.fit_one_cycle(
    64,
    max_lr=.01,
    wd=0.,
    pct_start=0.0,
    div_factor=100,
    callbacks=[logger, SaveModelCallback(learn, monitor='metric_tot', mode='max', name='new_baseline_Seresnext50MishSumSpatial_lastBN_LessAugs_211_Mu10_Wd0_64epochs'), MixUpCallback(learn, alpha=1.)]
)

epoch,train_loss,valid_loss,metric_grapheme,metric_vowel,metric_consonant,metric_tot,time
0,1.451362,0.264035,0.885149,0.955873,0.935986,0.915539,10:17
1,1.209941,0.214504,0.914991,0.967068,0.949544,0.936648,10:13
2,1.084246,0.153576,0.941143,0.975084,0.956667,0.953509,10:12
3,0.976826,0.141011,0.941426,0.973102,0.965669,0.955406,10:12
4,0.911649,0.130374,0.946029,0.980038,0.966104,0.959550,10:12
5,0.847235,0.124336,0.950107,0.980775,0.967562,0.962138,10:12
6,0.824942,0.126695,0.948181,0.980570,0.965267,0.960550,10:14
7,0.791352,0.117739,0.950849,0.981065,0.966583,0.962337,10:17
8,0.760089,0.114488,0.950906,0.982244,0.972122,0.964045,10:12
9,0.747264,0.113068,0.954787,0.981462,0.963045,0.963520,10:12
